After several crash we try something more simple, each cell constitute a small test 
- This is not complete and the final last function will be tested on Monday 27 of May 
- The tested sections are safe and you can run them as much as you want 
- you can try the untested cells... at your own risk . 

PART 1 : verification, just to make sure it works with the command, the rest of the code will be added bit by bit 

--- TESTED AND WORKS

In [4]:
# Import the propar module
import propar

# Connect to the local instrument.
el_flow = propar.instrument('COM4')

# Use the get_nodes function of the master of the instrument to get a list of instruments on the network
nodes = el_flow.master.get_nodes()

# Display the list of nodes
for node in nodes:
  print(node)

{'address': 4, 'type': 'DMFC', 'serial': 'M17205406A', 'id': '\x07SNM17205406A', 'channels': 1}
{'address': 5, 'type': 'DMFC', 'serial': 'M17205406D', 'id': '\x07SNM17205406D', 'channels': 1}


PART 2 : INPUT/OUTPUT
--- TESTED AN WORKS

In [5]:
import serial
import propar

user_input = input('Do you want to connect to the flow controller? (Y/N) ').upper()

if user_input == 'Y':
    # Connect to the local instrument
    el_flow = propar.instrument('COM4')

    # Use the get_nodes function of the master of the instrument to get a list of instruments on the network
    nodes = el_flow.master.get_nodes()

    # Display the list of nodes
    for node in nodes:
        print(node)

    

else:
    print('Exiting program.')

{'address': 4, 'type': 'DMFC', 'serial': 'M17205406A', 'id': '\x07SNM17205406A', 'channels': 1}
{'address': 5, 'type': 'DMFC', 'serial': 'M17205406D', 'id': '\x07SNM17205406D', 'channels': 1}


In [9]:
# Import the propar module
import propar

# Connect to the local instrument, when no settings provided
# defaults to locally connected instrument (address=0x80, baudrate=38400)
instrument = propar.instrument('COM4')

# Measure and setpoint in output units.
instrument.writeParameter(206, 0.0)
print(instrument.readParameter(205), instrument.readParameter(129)) # fMeasure, Capacity Unit

# Measure and setpoint scaled to 0-32000 = 0-100%
instrument.writeParameter(9, 16000)
print(instrument.readParameter(8))

# Measure and setpoint as property (also 0-32000)
instrument.setpoint = 0
print(instrument.measure)

0.034682728350162506 mln/min
26
37


PART 3 : TESTED AND WORKS (NUMPY FOR SIMPLICITY HERE)

In [12]:
import propar
import time
import numpy as np

# Function to read the temperature
def read_temperature(instrument):
    # Define the parameters to read the temperature
    params = [{'proc_nr': 33, 'parm_nr': 7, 'parm_type': propar.PP_TYPE_FLOAT}]
    
    # Read the parameters
    values = instrument.read_parameters(params)
    
    # Return the temperature value
    return values[0]['data']

# Main function
def main():
    # Initialize the instrument
    instrument = propar.instrument('COM4')
    
    # Ask the user for the duration of the experiment
    print('Enter the duration of the experiment in seconds:')
    duration = int(input('Duration in s: '))
    
    # Initialize a list to store temperature data
    temperature_data = []
    
    # Calculate the end time of the experiment
    end_time = time.time() + duration
    
    # Collect temperature data until the duration is over
    while time.time() < end_time:
        # Read the temperature
        temperature = read_temperature(instrument)
        
        # Append the temperature to the list
        temperature_data.append(temperature)
        
        # Print the temperature value
        print(f'Temperature: {temperature} °C')
        
        # Wait for a short interval before the next reading
        time.sleep(1)
    
    # Convert the list to a NumPy array
    temperature_array = np.array(temperature_data)
    
    # Print the final temperature array
    print('Temperature data collected:')
    print(temperature_array)

# Run the main function
if __name__ == "__main__":
    main()


Enter the duration of the experiment in seconds:
Temperature: 22.358232498168945 °C
Temperature: 22.361133575439453 °C
Temperature: 22.36322021484375 °C
Temperature: 22.367076873779297 °C
Temperature: 22.36941909790039 °C
Temperature: 22.37212371826172 °C
Temperature: 22.375858306884766 °C
Temperature: 22.382034301757812 °C
Temperature: 22.387807846069336 °C
Temperature: 22.389997482299805 °C
Temperature data collected:
[22.3582325  22.36113358 22.36322021 22.36707687 22.3694191  22.37212372
 22.37585831 22.3820343  22.38780785 22.38999748]


PART 4 : TESTED AND WORKS 
We just use xarray instead of numpy 

In [16]:
import propar
import time
import numpy as np
import xarray as xr

# Function to read the temperature
def read_temperature(instrument):
    # Define the parameters to read the temperature
    params = [{'proc_nr': 33, 'parm_nr': 7, 'parm_type': propar.PP_TYPE_FLOAT}]
    
    # Read the parameters
    values = instrument.read_parameters(params)
    
    # Return the temperature value
    return values[0]['data']

# Main function
def main():
    # Initialize the instrument
    instrument = propar.instrument('COM4')
    
    # Ask the user for the duration of the experiment
    print('Enter the duration of the experiment in seconds:')
    duration = int(input('Duration in s: '))
    
    # Initialize lists to store temperature data and timestamps
    temperature_data = []
    timestamps = []
    
    # Calculate the end time of the experiment
    end_time = time.time() + duration
    
    # Collect temperature data until the duration is over
    while time.time() < end_time:
        # Read the temperature
        temperature = read_temperature(instrument)
        
        # Append the temperature and timestamp to the lists
        temperature_data.append(temperature)
        timestamps.append(time.time())
        
        # Print the temperature value
        print(f'Temperature: {temperature} °C')
        
        # Wait for a short interval before the next reading
        time.sleep(1)
    
    # Convert the lists to xarray DataArray
    temperature_array = xr.DataArray(
        temperature_data,
        coords=[timestamps],
        dims=["time"],
        name="temperature"
    )
    
    # Print the final xarray DataArray
    print('Temperature data collected:')
    print(temperature_array)

# Run the main function
if __name__ == "__main__":
    main()


Enter the duration of the experiment in seconds:
Temperature: 24.664146423339844 °C
Temperature: 24.666156768798828 °C
Temperature: 24.66779136657715 °C
Temperature: 24.6721248626709 °C
Temperature: 24.6750545501709 °C
Temperature: 24.67704963684082 °C
Temperature: 24.680009841918945 °C
Temperature: 24.682016372680664 °C
Temperature: 24.682947158813477 °C
Temperature: 24.684785842895508 °C
Temperature data collected:
<xarray.DataArray 'temperature' (time: 10)> Size: 80B
array([24.66414642, 24.66615677, 24.66779137, 24.67212486, 24.67505455,
       24.67704964, 24.68000984, 24.68201637, 24.68294716, 24.68478584])
Coordinates:
  * time     (time) float64 80B 1.717e+09 1.717e+09 ... 1.717e+09 1.717e+09


PART 5 : We can read the temperature and store it ; Now we try to set a flow 

In [21]:
import propar

# Connect to the instrument
instrument = propar.instrument('COM4')

# Measure and setpoint in output units.
instrument.writeParameter(206, 0.0)
print(instrument.readParameter(205), instrument.readParameter(129)) # fMeasure, Capacity Unit

# Measure and setpoint scaled to 0-32000 = 0-100%
instrument.writeParameter(9, 16000)
print(instrument.readParameter(8))

# Measure and setpoint as property (also 0-32000)
instrument.setpoint = 0
print(instrument.measure)

0.02602020464837551 mln/min
27
36


THIS PART IS NOT COMPLETE , HERE IS THE CURRENT GOAL : 
- PART 5 is just from the bronkhorst polar github (with few adjustments)
- The goal was to test that our device can actually read and set temperature, it works and will be added to the main function 
- We will try add an input/output so the user can set his own flow rate. 
- Combined with the previous function, the user should be able to read the temperature with different flows. 
- -> Needs to be discussed on "how" ; should it be directly on this function or not. 
- Finally disconnecting the devices after the experiment is done 
- also we will implement the line of codes that print all the available devices INSIDE the main function 
- We limited the amount of input/output to make sure the device do not crash. 

In [30]:
import propar

# Connect to the instrument
instrument = propar.instrument('COM4')

# Define the maximum flow rate of the instrument (in output units, e.g., ml/min)
max_flow_rate = 100.0

# Prompt the user to enter the desired flow rate
desired_flow = float(input("Enter the desired flow rate (in output units): "))

# Ensure the desired flow rate does not exceed the maximum flow rate
if desired_flow > max_flow_rate:
    print(f"Error: Desired flow rate exceeds maximum flow rate of {max_flow_rate} {instrument.readParameter(129)}.")
else:
    # Set the flow rate in output units
    instrument.writeParameter(206, desired_flow)

    # Read and print the current flow rate and capacity unit
    current_flow = instrument.readParameter(205)
    capacity_unit = instrument.readParameter(129)
    print(f'Current flow rate: {current_flow} {capacity_unit}')

    # Optionally, work with scaled values (0-32000 = 0-100%)
    scaled_value = int((desired_flow / max_flow_rate) * 32000)  # Calculate scaled value
    instrument.writeParameter(9, scaled_value)

    # Read and print the scaled flow rate
    scaled_flow = i





Current flow rate: 0.025374120101332664 mln/min


PART X : 
We improve the past code. 
The approach is to have a dynamic programming way of doing things : so the user can select what he wants and needs. 
----- UPDATE : IT LEADS TO CRASH : DO NOT USE 

In [23]:
import time
import numpy as np


bronkhorst = propar.instrument('COM3')

# Initialize the data array
data = []

# Prompt the user to enter the duration in seconds
duration = float(input("Enter the duration in seconds: "))

# Record the start time
start_time = time.time()

# Run the loop for the specified duration
while time.time() - start_time < duration:
    # Read temperature and pressure values
    temperature = bronkhorst.readParameter(10)  # Replace 10 with the appropriate DDE parameter number for temperature
    pressure = bronkhorst.readParameter(11)  # Replace 11 with the appropriate DDE parameter number for pressure
    measure_value = bronkhorst.measure

    # Append the data to the NumPy array
    data.append([temperature, pressure, measure_value])

    # Optionally, add a small delay to avoid overwhelming the system with too many reads per second
    time.sleep(0.1)

# Convert the list to a NumPy array
data_array = np.array(data)

# Print the collected data
print(data_array)


[[    0 -7987     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7988     0]
 [    0 -7988     0]
 [    0 -7987     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7987     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7988     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7987     0]
 [    0 -7989     0]
 [    0 -7986     0]
 [    0 -7987     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7987     0]
 [    0 -7987     0]
 [    0 -7987     0]
 [    0 -7987     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7987     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7987     0]
 [    0 -7986     0]
 [    0 -7987     0]
 [    0 -7987     0]
 [    0 -7987     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7986     0]
 [    0 -7987     0]
 [    0 -7986

In [28]:
import time
import numpy as np
import propar

# Prompt the user to connect to the flow controller
user_input = input('Do you want to connect to the flow controller? (Y/N) ').upper()

if user_input == 'Y':
    # Connect to the local instrument
    el_flow = propar.instrument('COM3')

    # Prompt the user to set the duration in seconds
    duration = float(input("Enter the duration in seconds: "))

    # Prompt the user to set the baudrate
    print("Available baudrates: 9600, 19200, 38400, 57600, 115200")
    baudrate = int(input("Enter the desired baudrate: "))

    # Set the baudrate for the instrument
    el_flow.master.set_baudrate(baudrate)

    # Initialize the data array
    data = []

    # Record the start time
    start_time = time.time()

    # Run the loop for the specified duration
    while time.time() - start_time < duration:
        # Read temperature and pressure values
        temperature = el_flow.readParameter(10)  # Replace 10 with the appropriate DDE parameter number for temperature
        pressure = el_flow.readParameter(11)  # Replace 11 with the appropriate DDE parameter number for pressure
        measure_value = el_flow.measure

        # Append the data to the NumPy array
        data.append([temperature, pressure, measure_value])

        # Optionally, add a small delay to avoid overwhelming the system with too many reads per second
        time.sleep(0.1)

    # Convert the list to a NumPy array
    data_array = np.array(data)

    # Print the collected data
    print(data_array)

else:
    print('Exiting program.')

[[None None None]
 [None None None]
 [None None None]
 [None None None]
 [None None None]
 [None None None]
 [None None None]]
